**Processing Pipeline 2**

In [4]:
import os
import polars as pl
import pandas as pd
from datetime import datetime
import chardet

In [5]:
input_df = pl.read_csv("data/PP_ALL.csv", encoding="utf-8")
output_df = pl.read_csv("data/Joined_rtf_files.csv", encoding="utf-8")
input_df = input_df.with_columns(pl.col("Datum").str.strptime(pl.Datetime, "%m/%d/%Y %H:%M"))
output_df = output_df.with_columns(pl.col("Datum").str.strptime(pl.Datetime, "%m/%d/%Y %H:%M"))

FileNotFoundError: [Errno 2] No such file or directory: 'data\\PP_ALL.csv'